In [2]:
import pandas as pd
import statsmodels.api as sm
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import BayesianRidge, LinearRegression

In [3]:
test=pd.read_csv("C:\\Users\\HP\\Desktop\\testdata.csv")
train=pd.read_csv("C:\\Users\\HP\\Desktop\\traindata.csv")

In [4]:
train=train.drop(columns=['2019_Runs'])

In [5]:
train=train[['PLAYER', 'Mat', 'Inns', 'NO','HS', 'Avg', 'BF', 'SR',
       '100', '50', '4s', '6s','2018_Runs']]

In [6]:
for i in range(len(train)):
    k=str(train.iloc[i,4])
    if k[-1]=="*":
        train.iloc[i,4]=int(k[:-1])
train['HS']=np.asarray(train['HS'],dtype=float)
for i in range(len(train)):
    if train.iloc[i,5]=='-':
        train.iloc[i,5]=train.iloc[i,12]
train['Avg']=np.asarray(train['Avg'],dtype=float)

In [7]:
xtrain=train.iloc[:,1:-1]
ytrain=train.iloc[:,-1]

In [8]:
xtrain.columns

Index(['Mat', 'Inns', 'NO', 'HS', 'Avg', 'BF', 'SR', '100', '50', '4s', '6s'], dtype='object')

In [9]:
cols = list(xtrain.columns)
pmax = 1
while (len(cols)>0):
    p= []
    X_1 = xtrain[cols]
    X_1 = sm.add_constant(X_1)
    model = sm.OLS(ytrain,X_1).fit()
    p = pd.Series(model.pvalues.values[1:],index = cols)      
    pmax = max(p)
    feature_with_p_max = p.idxmax()
    if(pmax>0.05):
        cols.remove(feature_with_p_max)
    else:
        break
selected_features_BE = cols
print(selected_features_BE)

['NO', 'BF', 'SR', '50', '4s', '6s']


In [10]:
xtrain=xtrain[selected_features_BE]

In [11]:
outx=test.iloc[:,0]
xtest=test.drop(columns=['PLAYER','2019_Runs','HS','Avg','Mat','Inns','100'])

In [12]:
sc=StandardScaler()

In [13]:
xtrain=sc.fit_transform(xtrain)

In [14]:
xtest=sc.transform(xtest)

In [15]:
rr=RandomForestRegressor()

In [16]:
ytrain

0     134
1     480
2      63
3     370
4     148
     ... 
95    530
96     65
97    122
98    260
99     65
Name: 2018_Runs, Length: 100, dtype: int64

In [17]:
rr.fit(xtrain,ytrain)

RandomForestRegressor()

In [18]:
ypred=rr.predict(xtest)

In [19]:
outy=[]
for i in range(100):
    outy.append(int(round(ypred[i])))

In [20]:
data={'Player':outx,'2020_Runs':outy}

In [21]:
dfout=pd.DataFrame(data)

In [22]:
dfout

,Player,2020_Runs
0,David Warner,621
1,KL Rahul,599
2,Quinton de Kock,567
3,Shikhar Dhawan,506
4,Andre Russell,375
...,...,...
95,Carlos Brathwaite,17
96,Ishant Sharma,19
97,Shakib Al Hasan,16
98,Pawan Negi,15


In [23]:
dfout.to_csv('FliprHackathon.csv')